In [1]:

# ================================================
# 1️⃣ Env Prepare Install Required Packages
# ================================================

#%%capture
import os, importlib.util
!pip install --upgrade -qqq uv
if importlib.util.find_spec("torch") is None or "COLAB_" in "".join(os.environ.keys()):    
    try: import numpy, PIL; get_numpy = f"numpy=={numpy.__version__}"; get_pil = f"pillow=={PIL.__version__}"
    except: get_numpy = "numpy"; get_pil = "pillow"
    !uv pip install -qqq \
        "torch>=2.8.0" "triton>=3.4.0" {get_numpy} {get_pil} torchvision bitsandbytes "transformers==4.56.2" \
        "unsloth_zoo[base] @ git+https://github.com/unslothai/unsloth-zoo" \
        "unsloth[base] @ git+https://github.com/unslothai/unsloth" \
        git+https://github.com/triton-lang/triton.git@05b2c186c1b6c9a08375389d5efe9cb4c401c075#subdirectory=python/triton_kernels
elif importlib.util.find_spec("unsloth") is None:
    !uv pip install -qqq unsloth
!uv pip install --upgrade --no-deps transformers==4.56.2 tokenizers trl==0.22.2 unsloth unsloth_zoo huggingface-hub==0.34.4 datasets==4.3.0 numpy==2.3.4 pandas==2.3.3 pyarrow==22.0.0 tqdm==4.67.1

Using Python 3.12.11 environment at: /home/zeus/miniconda3/envs/cloudspace
Resolved 11 packages in 55ms                                         
Audited 11 packages in 0.25ms


In [2]:
import torch
import bitsandbytes as bnb
from collections import Counter
from unsloth import FastLanguageModel


model, tokenizer = FastLanguageModel.from_pretrained(
    "unsloth/gpt-oss-20b",
    load_in_4bit=True,
)

bnb_layers = []
for name, module in model.named_modules():
    if isinstance(module, bnb.nn.Linear4bit):
        bnb_layers.append(name)

print(f"BNB 4-bit layers found: {len(bnb_layers)}")
print("Sample BNB layer:", bnb_layers[0] if bnb_layers else "❌ NONE")
print("Model compute dtype:", model.dtype)



🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.12.10: Fast Gpt_Oss patching. Transformers: 4.56.2.
   \\   /|    NVIDIA H200. Num GPUs = 1. Max memory: 139.811 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu128. CUDA: 9.0. CUDA Toolkit: 12.8. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

BNB 4-bit layers found: 1632
Sample BNB layer: model.layers.0.self_attn.q_proj
Model compute dtype: torch.bfloat16


In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    checkpoint_path="ospost/gpt-oss-20b-sft-adapter"
)


Unsloth: Making `model.base_model.model.model` require gradients


In [4]:
assert any("lora_" in n for n,_ in model.named_parameters())

lora_params = [
    (n, p) for n, p in model.named_parameters() if "lora_" in n
]

print(f"LoRA parameters found: {len(lora_params)}")
print("Sample LoRA param:", lora_params[0][0] if lora_params else "❌ NONE")

print("LoRA dtypes:", Counter(p.dtype for _, p in lora_params))

trainable = [
    n for n, p in model.named_parameters() if p.requires_grad
]

print(f"Trainable params count: {len(trainable)}")
print("Sample trainable param:", trainable[0] if trainable else "❌ NONE")



LoRA parameters found: 192
Sample LoRA param: base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight
LoRA dtypes: Counter({torch.float32: 192})
Trainable params count: 192
Sample trainable param: base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight


In [5]:
model.save_pretrained_merged(
    "gpt-oss-20b-sft-mxfp4",
    tokenizer,
    save_method="mxfp4",
)


config.json: 0.00B [00:00, ?B/s]

Found HuggingFace hub cache directory: /teamspace/studios/this_studio/.cache/huggingface/hub


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Checking cache directory for required files...
Cache check failed: model-00000-of-00002.safetensors not found in local cache.
Not all required files found in cache. Will proceed with downloading.
Checking cache directory for required files...
Cache check failed: tokenizer.model not found in local cache.
Not all required files found in cache. Will proceed with downloading.


Unsloth: Preparing safetensor model files:   0%|          | 0/3 [00:00<?, ?it/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/4.17G [00:00<?, ?B/s]

Unsloth: Preparing safetensor model files: 100%|██████████| 3/3 [00:13<00:00,  4.34s/it]


Note: tokenizer.model not found (this is OK for non-SentencePiece models)


Unsloth: Merging weights into mxfp4: 100%|██████████| 3/3 [00:14<00:00,  4.70s/it]


Unsloth: Merge process complete. Saved to `/teamspace/studios/this_studio/gpt-oss-20b-fine-tune/gpt-oss-20b-sft-mxfp4`


In [ ]:
!ls -lah gpt-oss-20b-sft-mxfp4

In [ ]:
model.save_pretrained_merged(
    "gpt-oss-20b-sft-bf16",
    tokenizer
)


In [ ]:
!ls -lah gpt-oss-20b-sft-bf16

In [ ]:
# model.push_to_hub("ospost/gpt-oss-20b-sft-mxfp4", token = "")


In [ ]:
# git clone https://github.com/ggerganov/llama.cpp
# cd llama.cpp

# python convert_hf_to_gguf.py \
#  gpt-oss-20b-sft-mxfp4 \
#  --outtype q4_K_M \
#  --outfile gpt-oss-20b-sft-q4_K_M.gguf
